## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [1]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict

from string import punctuation
import re

# Feel free to include your text patterns functions
# Stopwords
sw = nltk.corpus.stopwords.words("english")
#Punctuation
punctuation = set(punctuation)
def clean_text_data(text):
    new_text = text
    for i in new_text:
        if i in punctuation:
            new_text = new_text.replace(i, " ")
    # Remove extra white space
    new_text = re.sub(r'\s+', ' ', new_text)
    # Split on whitespace
    new_text = new_text.split()
    # Fold to lowercase
    for i in range(len(new_text)):
        new_text[i] = new_text[i].lower()
    # Remove stopwords
    new_text = [i for i in new_text if i not in sw]
    # Remove blank items
    new_text = [j for j in new_text if j]
    return new_text
#from text_functions_solutions import clean_tokenize, get_patterns

In [2]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [3]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

query_results = convention_cur.execute(
                            '''
                            SELECT party, text
                            FROM conventions
                            ''')
for row in query_results :
    # store the results in convention_data
    convention_data.append([' '.join(clean_text_data(row[1])), row[0]])    

Let's look at some random entries and see if they look right. 

In [4]:
random.choices(convention_data,k=10)

[['military better equipped better resourced better manned military world president trump demolished terrorist isis caliphate middle east eliminated leader al baghdadi one world’s brutal terrorists president trump took decisive action iranian terrorist mastermind qasem soleimani man responsible deaths hundreds american service men iraq nato allies failed meet commitments upheld president trump demanded parody nato members increased contributions 100 billion year nato’s secretary general credits president donald j trump',
  'Republican'],
 ['help god', 'Republican'],
 ['lost nearly 50 business china mid 2000s wondered small company like could continue compete time joe biden senator voted normalize trade china help pave way join wto even though hurting american companies like we’re tenacious we’re creative took risk purchased 3d printer 3d printing technology allows us things china can’t',
  'Republican'],
 ['protests st louis turn violent', 'Republican'],
 ['i’m example woman given seco

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [5]:
word_cutoff = 10

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 10, we have 1478 as features in the model.


In [6]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Your code here
    ret_dict = dict()
    for i in text.split():
        if i in fw:
            ret_dict[i] = True
    return(ret_dict)

In [7]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [8]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [9]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [10]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.492


In [11]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 defense = True           Republ : Democr =     14.0 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                    isis = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

Most of the informative features are weighted heavier Republican than Democrat, indicating that there are more words that distinguish an individual as a Republican than as a Democrat. The words themselves also are interesting as they can reveal policy positions and issues that the respective party finds most important. For example, the word "china" was used far more frequently by Republicans than Democrats, while "votes" was used more frequently by Democrats compared to Republicans. Overall, this analysis can help reveal what each party finds to be the most important issues of the day. 


## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [12]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [13]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [14]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.
for row in results:
    tweet_data.append([' '.join(clean_text_data(row[2].decode("utf-8", "ignore"))), row[1]]) 

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [15]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [16]:
for tweet, party in tweet_data_sample :
    estimated_party = classifier.classify(conv_features(tweet, feature_words))
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")

Here's our (cleaned) tweet: earlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast https co wqgtrzt7vv
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: go tribe rallytogether https co 0nxutfl9l5
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: apparently trump thinks easy students overwhelmed crushing burden debt pay student loans trumpbudget https co ckyqo5t0qh
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: we’re grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide much needed help putting lives line https co ezpv0vmiz3
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: let’s make even greater kag 🇺🇸 https co y9qozd5l2z
Actual party is Republican and our classifer says Democratic.

Here's our (cleaned) tweet:

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [17]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = classifier.classify(conv_features(tweet, feature_words))
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [18]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3104, 'Democratic': 1174}),
             'Democratic': defaultdict(int,
                         {'Republican': 4075, 'Democratic': 1649})})

### Reflections

The primary issue with the model is it primarily predicts Republican, even for Democratic tweets. This could be centered around the previous comment, that Republicans have more distinguishing words than Democrats in speeches. Since a word appears to be more likely to indicate an individual is Republican, the model overpredicts Republican. Naive Bayes may not be the best model for this data. It may also be useful to gather more speech data to train the data on, as it may reveal more Democratic weighted words. Data from previous conventions, for example, may be more insightful and produce more accurate results. The word usage and content of speeches would likely be impacted by which party currently controlled the White House, the House, and the Senate. Current political climate, world issues, and more would also likely impact the word choice in speeches. Therefore, selecting only one year may give the best representation of word choice by party. It should also be considered that conditions in 2020 were vastly different from those in 2018 due simply to the COVID-19 pandemic. This would surely impact word choice as well.